In [7]:
import ast
import pandas as pd
import numpy as np
import pysam
import os
import matplotlib.pyplot as plt

# Heng Li

In [8]:
directory = '/home/mark/Desktop/HLA/HengLi/Runs/'
boundaries={}
haplotypes=[]
hengCoordinates={}
for folder in os.listdir(directory):
    boundaries[folder.split(".")[0]+".hap"+str(folder.split(".")[1])]={'Start':0, 'End':0}
    haplotypes.append(folder.split(".")[0]+".hap"+str(folder.split(".")[1]))
    df = pd.read_csv(directory+str(folder), comment='#', sep='\t', header=None)
    goodRows=[]
    for row in df.index:
        if 'HLA-DRB' in str(df.at[row,8]):
             goodRows.append(row)
        else:
            continue
    df2 = df.loc[goodRows].copy()
    df3 = df2[df2[2]=='exon'].copy()
    df3[3] = df3[3].astype(int)
    df3[4] = df3[4].astype(int)
    df3.sort_values(by=[3], inplace=True)
    count=len(df3)
    start=min(df3[3])
    end=max(df3[4])
    tempList=[]
    for row in df3.index:
        tempList.append([x for x in df3.loc[row]])
    hengCoordinates[folder.split(".")[0]+".hap"+str(folder.split(".")[1])]={'Start':start, 'End':end, 'Total_Genes':count, 'Exon_List':tempList}

In [9]:
#THIS CODE WILL GRAB THE BOUNDARIES OF DRA - DQA1
directory = '/home/mark/Desktop/HLA/HengLi/Runs/'

for folder in os.listdir(directory):

    df = pd.read_csv(directory+str(folder), comment='#', sep='\t', header=None)
    goodDRA=[]
    goodDQA1=[]
    for row in df.index:
        if 'HLA-DRA' in str(df.at[row,8]):
             goodDRA.append(row)
        elif 'HLA-DQA1' in str(df.at[row,8]):
             goodDQA1.append(row)
        
        else:
            continue
    df2 = df.loc[goodDRA].copy()
    df3 = df2[df2[2]=='gene'].copy()
    df3[3] = df3[3].astype(int)
    df3[4] = df3[4].astype(int)
    df3.sort_values(by=[3], inplace=True)
    start=max(df3[4])
    
    df2_dqa1 = df.loc[goodDQA1].copy()
    df3_dqa1 = df2_dqa1[df2_dqa1[2]=='gene'].copy()
    df3_dqa1[3] = df3_dqa1[3].astype(int)
    df3_dqa1[4] = df3_dqa1[4].astype(int)
    df3_dqa1.sort_values(by=[3], inplace=True)
    end=min(df3_dqa1[3])

    hengCoordinates[folder.split(".")[0]+".hap"+str(folder.split(".")[1])]['Start']=start
    hengCoordinates[folder.split(".")[0]+".hap"+str(folder.split(".")[1])]['End']=end

# My data Run

In [11]:
haplotypes

['HG02666.hap2',
 'HG02059.hap2',
 'NA18534.hap1',
 'NA19240.hap1',
 'NA19317.hap1',
 'HG02818.hap1',
 'HG03683.hap2',
 'NA24385.hap1',
 'NA19836.hap1',
 'NA21487.hap1',
 'HG03683.hap1',
 'HG01890.hap1',
 'HG03456.hap2',
 'chm13.hap1',
 'HG02282.hap1',
 'HG00268.hap1',
 'NA12329.hap2',
 'HG00513.hap2',
 'HG03371.hap2',
 'HG00171.hap1',
 'NA19331.hap1',
 'NA19705.hap1',
 'NA19317.hap2',
 'NA12329.hap1',
 'NA19434.hap1',
 'HG00358.hap2',
 'HG00096.hap2',
 'HG00732.hap2',
 'HG03009.hap1',
 'HG01573.hap1',
 'HG00171.hap2',
 'HG00731.hap1',
 'NA19384.hap1',
 'HG01457.hap2',
 'HG02666.hap1',
 'HG00733.hap2',
 'HG03371.hap1',
 'HG03732.hap2',
 'NA19239.hap2',
 'NA20355.hap2',
 'HG02011.hap2',
 'NA19036.hap1',
 'NA19836.hap2',
 'NA19705.hap2',
 'hg38.hap8',
 'HG00513.hap1',
 'HG01505.hap1',
 'HG00512.hap2',
 'HG02953.hap2',
 'HG03807.hap1',
 'HG01457.hap1',
 'HG02018.hap1',
 'HG02587.hap2',
 'NA18534.hap2',
 'HG03456.hap1',
 'HG03248.hap1',
 'NA19347.hap1',
 'HG02587.hap1',
 'NA19036.hap2',
 '

In [12]:
#THIS SHOULD SAY 130
print(len(haplotypes))

137


In [17]:
import os
repeatDict={}
for file in os.listdir('/home/mark/Desktop/HLA/repeatmasker/plain_RepeatMasker/'):
    repeatDict[str(file.split(".")[0])+"."+str(file.split(".asm-")[1].split(".")[0])]='/home/mark/Desktop/HLA/repeatmasker/plain_RepeatMasker/'+str(file)

In [18]:
for x in repeatDict:
    if 'hg38' in str(x):
        print(x)

hg38.hap3
hg38.hap5
hg38.hap4
hg38.hap6
hg38.hap7
hg38.hap8
hg38.hap2
hg38.hap1


In [19]:
for haplotype in set(haplotypes):
    
    drb_rm_list=[]
    minimum= int(hengCoordinates[haplotype]['Start'])
    maximum= int(hengCoordinates[haplotype]['End'])
    
    with open(repeatDict[haplotype], 'r') as file:
        for line in file:
            goodline = ','.join(line.split())
            if '.hap' in str(goodline) and '*' not in str(goodline):
                startrm = int(goodline.split(",")[5])
                endrm = int(goodline.split(",")[6])

                if startrm >= minimum and startrm <= maximum:
                    tempList2 = [x for x in goodline.split(",")]
                    tempList2.append(str(minimum))
                    tempList2.append(str(startrm-minimum))
                    drb_rm_list.append(tempList2)
                else:
                    continue
                        
        finalDRB = pd.DataFrame(data=drb_rm_list, columns=['Score','Divergence','Ins','Del','Hap','Start','End','Length','Orientation', 'Annotation', 'Element_Group', 'pos1', 'pos2', 'pos3','unimportant','Coordinate_Start','Distance_From_Start'])
        finalDRB['UniqueName']=[str(finalDRB.at[x,'Hap'])+"_"+str(finalDRB.at[x,'Start'])+"_"+str(finalDRB.at[x,'Annotation']) for x in finalDRB.index]
        finalDRB.set_index("UniqueName", inplace=True)
            
    file.close()
        
    drb_rm_list2=[]
    
    for item in hengCoordinates[haplotype]['Exon_List']:
        drb_rm_list2.append([haplotype, item[3], item[4], item[6], minimum, int(item[3])-minimum, item[8].split("gene_name")[1].split(";")[0], item[8].split("exon_number")[1].split(";")[0]])
    
    finalDRB2 = pd.DataFrame(data=drb_rm_list2, columns=['Hap','Start','End','Orientation','Coordinate_Start','Distance_From_Start', 'Gene_Name', 'Exon_Number'])
    finalDRB2['UniqueName']=[str(finalDRB2.at[x,'Hap'])+"_"+str(finalDRB2.at[x,'Start'])+"_"+str(finalDRB2.at[x,'Gene_Name']) for x in finalDRB2.index]
    finalDRB2.set_index("UniqueName", inplace=True)
        
    finalDF = pd.concat([finalDRB, finalDRB2])
    finalDF['Start']= finalDF['Start'].astype(int)
    finalDF['End']= finalDF['End'].astype(int)
    finalDF.sort_values(by=['Start'], inplace=True)
        
    finalDF.to_csv("/home/mark/Desktop/HLA/DRB/DataFrames/full/"+str(haplotype)+".csv")


In [20]:
repeatDict2={x:repeatDict[x].split("/plain_RepeatMasker/")[1].split(".out")[0] for x in repeatDict.keys()}

In [21]:
repeatDict2

{'HG03807.hap2': 'HG03807.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG01114.hap1': 'HG01114.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG00171.hap2': 'HG00171.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'NA19434.hap2': 'NA19434.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG01352.hap2': 'HG01352.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG03371.hap2': 'HG03371.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG03683.hap1': 'HG03683.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG03683.hap2': 'HG03683.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG00733.hap1': 'HG00733.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG00514.hap2': 'HG00514.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG00732.hap1': 'HG00732.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'NA19036.hap1': 'NA19036.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG00512.hap1': 'HG00512.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG02818.unassigned': 'HG02818.vrk-ps-sseq.asm-unassigned.HLA.cleaned.fasta',
 'NA18939.unassigned': 'NA18939.vrk-ps-sseq.asm-un

In [23]:
lengthMatch=[]
for haplotype in haplotypes:
    drb_rm_list=[]
    drb_rm_list2=[]
    
    minimum= int(hengCoordinates[haplotype]['Start'])
    maximum= int(hengCoordinates[haplotype]['End'])
    
    lengthMatch.append([haplotype, maximum-minimum])
    
    sequence = pysam.faidx('/home/mark/Desktop/HLA/Globus/cleaned/'+str(repeatDict2[haplotype]), haplotype+":"+str(minimum)+"-"+str(maximum))
    #with open('/home/mark/Desktop/HLA/DRB/Sequences/Individual/'+str(haplotype)+".fasta", 'a+') as file:
    #    file.write(sequence)
    #file.close()